In [ ]:
"""
Goal: hourly power breakdown for South Korea from Jan 1st, 2019 to December 31st, 2019

Output csv format:

datetime, 
timestamp,
zone_name,
total_consumption_avg,
power_production_biomass_avg,
power_production_coal_avg,
power_production_gas_avg,
power_production_hydro_avg,
power_production_nuclear_avg,
power_production_oil_avg,
power_production_solar_avg,
power_production_wind_avg,
power_production_geothermal_avg,
power_production_unknown_avg
"""
